# Exercises XP Ninja: W1_D4

## What You Will Learn

In this exercise, you will implement **Conway's Game of Life** using Python classes.  
You will learn how to model a grid, apply iterative rules to transform its state, and display the results over multiple generations.  
This will help you practice object-oriented design, loops, and conditional logic.

---

### Exercise 1: Conway's Game of Life

#### Rules (Wikipedia)

- Any live cell with fewer than two live neighbours dies (underpopulation).
- Any live cell with two or three live neighbours lives on to the next generation.
- Any live cell with more than three live neighbours dies (overpopulation).
- Any dead cell with exactly three live neighbours becomes a live cell (reproduction).

---

### Requirements

- The universe is a fixed-size two-dimensional grid (for the first version).
- Each cell is either **alive** (1) or **dead** (0).
- Each step (generation) is computed based on the current grid using the rules above.
- The grid should be displayed after each generation.
- The end state of the game is determined entirely by the initial state.
- Use **classes** to structure your code.
- First implement **fixed borders**: cells moving out of the border are considered dead.

---

### Bonus

- Implement a version with **expandable borders**.
- If a live cell touches the edge, add a new row or column in that direction.
- Use a large maximum size (e.g., 10,000) to prevent memory overflow.

## Conway's Game of Life — Core Class (fixed borders)

In [1]:
# Title: Conway's Game of Life — Core Class (fixed borders)
# This implementation uses a 2D grid with fixed borders (outside cells are treated as dead).
# Display uses ASCII: '█' for alive, '·' for dead.

from typing import List, Tuple, Iterable
import copy

Cell = int  # 0 = dead, 1 = alive
Grid = List[List[Cell]]

class GameOfLife:
    def __init__(self, rows: int, cols: int, seed: Iterable[Tuple[int, int]] | None = None):
        """
        Initialize a rows x cols grid. Optionally seed with a list of (r, c) coordinates set to alive (1).
        Fixed borders: any neighbor off-grid is considered dead (0).
        """
        if rows <= 0 or cols <= 0:
            raise ValueError("rows and cols must be positive.")
        self.rows = rows
        self.cols = cols
        self.grid: Grid = [[0 for _ in range(cols)] for _ in range(rows)]
        if seed:
            for (r, c) in seed:
                if 0 <= r < rows and 0 <= c < cols:
                    self.grid[r][c] = 1  # set alive

    def count_live_neighbors(self, r: int, c: int) -> int:
        """Count live neighbors around (r, c) considering 8 directions; off-grid cells are dead."""
        total = 0
        for dr in (-1, 0, 1):
            for dc in (-1, 0, 1):
                if dr == 0 and dc == 0:
                    continue
                rr, cc = r + dr, c + dc
                if 0 <= rr < self.rows and 0 <= cc < self.cols:
                    total += self.grid[rr][cc]
        return total

    def step(self) -> None:
        """
        Advance the simulation by one generation following Conway's rules:
        - Live cell with <2 live neighbors dies (underpopulation)
        - Live cell with 2 or 3 live neighbors lives
        - Live cell with >3 live neighbors dies (overpopulation)
        - Dead cell with exactly 3 live neighbors becomes alive (reproduction)
        """
        next_grid = copy.deepcopy(self.grid)
        for r in range(self.rows):
            for c in range(self.cols):
                live_neighbors = self.count_live_neighbors(r, c)
                if self.grid[r][c] == 1:
                    # Currently alive
                    if live_neighbors < 2 or live_neighbors > 3:
                        next_grid[r][c] = 0
                    else:
                        next_grid[r][c] = 1
                else:
                    # Currently dead
                    if live_neighbors == 3:
                        next_grid[r][c] = 1
                    else:
                        next_grid[r][c] = 0
        self.grid = next_grid

    def display(self) -> None:
        """Print the current grid using ASCII characters."""
        alive, dead = "█", "·"
        for r in range(self.rows):
            print("".join(alive if self.grid[r][c] == 1 else dead for c in range(self.cols)))
        print()  # blank line for readability

    def run(self, generations: int, display_each: bool = True) -> None:
        """Run the simulation for a given number of generations, optionally displaying each generation."""
        if display_each:
            print("Generation 0:")
            self.display()
        for gen in range(1, generations + 1):
            self.step()
            if display_each:
                print(f"Generation {gen}:")
                self.display()

## Helper Seeds — classic patterns (within a 10x10 default board)

In [2]:
# We define a few classic seeds: Block (still life), Blinker (oscillator), Glider (spaceship).

def seed_block() -> list[tuple[int, int]]:
    # 2x2 block at roughly center
    return [(4,4), (4,5), (5,4), (5,5)]

def seed_blinker() -> list[tuple[int, int]]:
    # Horizontal line of 3
    return [(4,3), (4,4), (4,5)]

def seed_glider() -> list[tuple[int, int]]:
    # Small glider (top-left)
    return [(1,2), (2,3), (3,1), (3,2), (3,3)]

## Demo — Run a few seeds (fixed borders)

In [3]:
# You can change rows/cols and the seed if you like.

# Block: should remain the same (still life)
game_block = GameOfLife(rows=10, cols=10, seed=seed_block())
game_block.run(generations=4, display_each=True)

# Blinker: should flip orientation every generation (period 2)
game_blinker = GameOfLife(rows=10, cols=10, seed=seed_blinker())
game_blinker.run(generations=4, display_each=True)

# Glider: moves diagonally down-right until it hits the border (fixed borders stop it)
game_glider = GameOfLife(rows=10, cols=10, seed=seed_glider())
game_glider.run(generations=6, display_each=True)

Generation 0:
··········
··········
··········
··········
····██····
····██····
··········
··········
··········
··········

Generation 1:
··········
··········
··········
··········
····██····
····██····
··········
··········
··········
··········

Generation 2:
··········
··········
··········
··········
····██····
····██····
··········
··········
··········
··········

Generation 3:
··········
··········
··········
··········
····██····
····██····
··········
··········
··········
··········

Generation 4:
··········
··········
··········
··········
····██····
····██····
··········
··········
··········
··········

Generation 0:
··········
··········
··········
··········
···███····
··········
··········
··········
··········
··········

Generation 1:
··········
··········
··········
····█·····
····█·····
····█·····
··········
··········
··········
··········

Generation 2:
··········
··········
··········
··········
···███····
··········
··········
··········
··········
··········



## Bonus — Expandable borders variant

In [4]:
# We subclass GameOfLife to auto-pad the grid when live cells reach edges, up to max_size.

class ExpandingGameOfLife(GameOfLife):
    def __init__(self, rows: int, cols: int, seed=None, max_size: int = 10000):
        super().__init__(rows, cols, seed)
        self.max_size = max_size

    def _needs_padding(self) -> tuple[bool, bool, bool, bool]:
        """
        Return which borders need padding: (top, bottom, left, right)
        if there is any live cell on that border.
        """
        top = any(self.grid[0][c] == 1 for c in range(self.cols))
        bottom = any(self.grid[self.rows-1][c] == 1 for c in range(self.cols))
        left = any(self.grid[r][0] == 1 for r in range(self.rows))
        right = any(self.grid[r][self.cols-1] == 1 for r in range(self.rows))
        return top, bottom, left, right

    def _pad(self, top: bool, bottom: bool, left: bool, right: bool) -> None:
        """Pad the grid with dead rows/cols if required, respecting max_size."""
        add_rows = (1 if top else 0) + (1 if bottom else 0)
        add_cols = (1 if left else 0) + (1 if right else 0)
        new_rows = self.rows + add_rows
        new_cols = self.cols + add_cols

        if new_rows > self.max_size or new_cols > self.max_size:
            # Do not expand beyond limits
            return

        new_grid = [[0 for _ in range(new_cols)] for _ in range(new_rows)]

        # Offsets after padding
        r_off = 1 if top else 0
        c_off = 1 if left else 0

        # Copy old grid into new grid
        for r in range(self.rows):
            for c in range(self.cols):
                new_grid[r + r_off][c + c_off] = self.grid[r][c]

        # Update dimensions and grid
        self.grid = new_grid
        self.rows = new_rows
        self.cols = new_cols

    def step(self) -> None:
        """Perform one step; then expand borders if needed."""
        super().step()
        top, bottom, left, right = self._needs_padding()
        if any((top, bottom, left, right)):
            self._pad(top, bottom, left, right)

## Demo — Expanding borders with a glider near the edge

In [6]:
seed = [(0,1), (1,2), (2,0), (2,1), (2,2)]  # glider touching the top-left
exp_game = ExpandingGameOfLife(rows=5, cols=5, seed=seed, max_size=200)
exp_game.run(generations=10, display_each=True)

Generation 0:
·█···
··█··
███··
·····
·····

Generation 1:
······
·█·█··
··██··
··█···
······

Generation 2:
······
···█··
·█·█··
··██··
······

Generation 3:
······
··█···
···██·
··██··
······

Generation 4:
······
···█··
····█·
··███·
······

Generation 5:
······
······
··█·█·
···██·
···█··
······

Generation 6:
······
······
····█·
··█·█·
···██·
······

Generation 7:
·······
·······
···█···
····██·
···██··
·······

Generation 8:
·······
·······
····█··
·····█·
···███·
·······

Generation 9:
·······
·······
·······
···█·█·
····██·
····█··
·······

Generation 10:
·······
·······
·······
·····█·
···█·█·
····██·
·······



## Conclusion

In this exercise, I implemented Conway's Game of Life using Python classes.

I practiced:
- Representing a grid with nested lists and managing fixed borders.
- Counting neighbours in all eight directions.
- Applying the four Game of Life rules to produce the next generation.
- Using **loops** and **conditional statements** to update the game state.
- Displaying each generation in a clear ASCII format.
- (Bonus) Expanding the grid dynamically when live cells reach the borders.

This project helped me reinforce my understanding of class design, iterative algorithms, and working with two-dimensional data structures in Python.

In [9]:
import time
from IPython.display import clear_output

class GameOfLife:
    def __init__(self, rows: int, cols: int, seed: Iterable[Tuple[int, int]] | None = None):
        if rows <= 0 or cols <= 0:
            raise ValueError("rows and cols must be positive.")
        self.rows = rows
        self.cols = cols
        self.grid: Grid = [[0 for _ in range(cols)] for _ in range(rows)]
        if seed:
            for (r, c) in seed:
                if 0 <= r < rows and 0 <= c < cols:
                    self.grid[r][c] = 1

    def count_live_neighbors(self, r: int, c: int) -> int:
        total = 0
        for dr in (-1, 0, 1):
            for dc in (-1, 0, 1):
                if dr == 0 and dc == 0:
                    continue
                rr, cc = r + dr, c + dc
                if 0 <= rr < self.rows and 0 <= cc < self.cols:
                    total += self.grid[rr][cc]
        return total

    def step(self) -> None:
        next_grid = [[0 for _ in range(self.cols)] for _ in range(self.rows)]
        for r in range(self.rows):
            for c in range(self.cols):
                live_neighbors = self.count_live_neighbors(r, c)
                if self.grid[r][c] == 1:
                    if live_neighbors in (2, 3):
                        next_grid[r][c] = 1
                else:
                    if live_neighbors == 3:
                        next_grid[r][c] = 1
        self.grid = next_grid

    def display(self) -> None:
        alive, dead = "█", "·"
        for r in range(self.rows):
            print("".join(alive if self.grid[r][c] == 1 else dead for c in range(self.cols)))
        print()

    def run(self, generations: int, display_each: bool = True, delay: float = 0.3) -> None:
        """Run the simulation for a given number of generations with optional animation delay."""
        for gen in range(generations + 1):
            if display_each:
                clear_output(wait=True)
                print(f"Generation {gen}:")
                self.display()
                time.sleep(delay)
            if gen < generations:
                self.step()


In [10]:
game = GameOfLife(rows=10, cols=10, seed=seed_glider())
game.run(generations=20, display_each=True, delay=0.2)

Generation 20:
··········
··········
··········
··········
··········
··········
·······█··
········█·
······███·
··········

